# Deep Learning with TensorFlow - Creating the Recurrent Neural Network (RNN) Model

Recurrent Neural Networks (RNNs) are popular models that have shown great promise in many NLP tasks. The Recurrent Neural Network attempts to address the necessity of understanding data in sequences. In this tutorial we'll work with MNIST dataset. MNIST is a simple computer vision dataset. It consists of images of handwritten digits like the image below. We will then train a RNN model on the training set using TensorFlow and make predictions on a test set.


![](https://www.tensorflow.org/images/MNIST.png)

Resources:

- [About MNIST](https://www.tensorflow.org/get_started/mnist/beginners)
- [My basic Neural Network project on this dataset](http://suruchifialoke.com/2017-06-15-predicting-digits_tensorflow/)
- [Tutorial by PythonProgramming.net](https://pythonprogramming.net/recurrent-neural-network-rnn-lstm-machine-learning-tutorial/)
- [Tutorial by WildML](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)
- [A research paper on RNN, that I found useful](http://www.aclweb.org/anthology/P14-1140.pdf)


## What are RNNs



The idea behind RNNs is to make use of sequential information. In a traditional neural network we assume that all inputs (and outputs) are independent of each other. But for many tasks that is very limiting. If we want to predict the next word in a sentence we can perform better by knowing which words came before it. RNNs are called recurrent because they perform the same task for every element of a sequence, with the output being depended on the previous computations. 

RNNs can make use of information in arbitrarily long sequences, but in practice they are limited to looking back only a few steps. Here is what a typical RNN looks like:
![](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg)


The above diagram shows a RNN being unrolled (or unfolded) into a full network. By unrolling we  mean that we write out the network for the complete sequence. For example, if the sequence we care about is a sentence of 5 words, the network would be unrolled into a 5-layer neural network, one layer for each word. The formulas that govern the computation happening in a RNN are as follows:

- x_t is the input at time step t. For example, x_1 could be a one-hot vector corresponding to the second word of a sentence.
- s_t is the hidden state at time step t. It’s the “memory” of the network. s_t is calculated based on the previous hidden state and the input at the current step: s_t=f(Ux_t + Ws_{t-1}). The function f usually is a nonlinearity such as tanh or ReLU.  s_{-1}, which is required to calculate the first hidden state, is typically initialized to all zeroes.
- o_t is the output at step t. For example, if we wanted to predict the next word in a sentence it would be a vector of probabilities across our vocabulary. o_t = \mathrm{softmax}(Vs_t).

## Understanding and Loading the Data

We're going to be working first with the MNIST dataset, which is a dataset that contains 60,000 training samples and 10,000 testing samples of hand-written and labeled digits, 0 through 9, so ten total "classes." 

The MNIST dataset has the images (see example above), which we'll be working with as purely black and white, thresholded, images, of size 28 x 28, or 784 pixels total. 

![](https://www.tensorflow.org/images/MNIST-Matrix.png)

Our features will be the pixel values for each pixel, thresholded. Either the pixel is "blank" (nothing there, a 0), or there is something there (1). Those are our features. We're going to attempt to just use this extremely rudimentary data, and predict the number we're looking at (a 0,1,2,3,4,5,6,7,8, or 9).


In [ ]:
import tensorflow as tf
# loading the data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

The MNIST data is split into three parts: 55,000 data points of training data (mnist.train), 10,000 points of test data (mnist.test), and 5,000 points of validation data (mnist.validation). 

For the purposes of this tutorial, we're going to want our labels as "one-hot vectors". A one-hot vector is a vector which is 0 in most dimensions, and 1 in a single dimension. In this case, the nth digit will be represented as a vector which is 1 in the nth dimension. For example, 3 would be [0,0,0,1,0,0,0,0,0,0]. Consequently, mnist.train.labels is a [55000, 10] array of floats.

## Building the model: Setting up the Computation Model

- Here, we're importing the rnn model/cell code from TensorFlow. 
- We're also defining the chunk size, number of chunks, and rnn size as new variables.
- Also, the shape of the x variable to include the chunks. 

In the basic neural network, we were sending in the entire image of pixel data all at once. With the Recurrent Neural Network, we're treating inputs now as sequential inputs of chunks instead.

In [ ]:
# import rnn from tensorFlow
from tensorflow.contrib import rnn 

# define number of classes = 10 for digits 0 through 9 
n_classes = 10

# defining the chunk size, number of chunks, and rnn size as new variables
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128

# placeholders for variables x and y
x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

We have a weights/biases dictionary like before, but then we get to some modifications to our input data, x. We're doing this is purely to satisfy the structure that TensorFlow wants of us to fit their rnn_cell model. 

In [ ]:
def recurrent_neural_network(x):

    layer = {'weights': tf.Variable(tf.random_normal([rnn_size, n_classes])),
                      'biases': tf.Variable(tf.random_normal([n_classes]))}

    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)

    lstm_cell = rnn.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']

    return output



## Training the model

Under a new function, train_neural_network, we will pass our output data.

- We then produce a prediction based on the output of that data through our neural_network_model. 
- Next, we create a cost variable. This measures how wrong we are, and is the variable we desire to minimize by manipulating our weights. The cost function is synonymous with a loss function. 
- To optimize our cost, we will use the AdamOptimizer, which is a popular optimizer along with others like Stochastic Gradient Descent and AdaGrad, for example.
- Within AdamOptimizer(), we can optionally specify the learning_rate as a parameter. The default is 0.001, which is fine for most circumstances. 
- Now that we have these things defined, we begin the session.

In [ ]:
hm_epochs = 4

def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

    #optimizer is learning rate but in this casde the default s fine
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            #_ is variable we dont care about
            #we have total number items and divide by batch_size for number of batches
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                #c is cost
                _, c = sess.run([optimizer, cost], feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss += c

            print('Epoch', epoch, ' completed out of ', hm_epochs, ' loss: ', epoch_loss)

        #we come here after optimizing the weights
        #tf.argmax is going to return the index of max values
        #in this training module we are checking if the prediction is matching the value
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        #accuracy is the float value of correct
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        #this is just evaluation
        print('Accuracy: ', accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))

train_neural_network(x)

Somewhere between 10 and 20 epochs should give us ~95% accuracy. 95% accuracy, sounds great, but is actually considered to be very bad compared to more popular methods. Consider that the only information we gave to our network was pixel values, that's it. We did not tell it about looking for patterns, or how to tell a 4 from a 9, or a 1 from a 8. The network simply figured it out with an inner model, based purely on pixel values to start, and achieved 93% accuracy in just 4 epochs.